In [1]:
import FinanceDataReader as fdr
import pandas as pd
import FinanceDataReader as fdr
from datetime import datetime, timedelta
import warnings

from pykrx import stock
from pykrx import bond
from defs_pred import filter_df
from defs import filter_df
from PublicDataReader import Fred
from marcap import marcap_data
from functools import reduce

In [2]:
targer_day = datetime(year=2024,month=2,day=15)
end_info_day = (targer_day - timedelta(days=1))
delta = timedelta(days=210)
day_120 = (end_info_day - delta).strftime("%Y-%m-%d")

warnings.simplefilter(action='ignore', category=FutureWarning) # FutureWarning 제거

# 코스피 지수 
KSI_df = fdr.DataReader('KS11','2020-01-01','2024-02-01').reset_index().drop(
    ['Open','High','Low','Adj Close'], axis=1).rename(
        columns={'Close':'KSI_Clo','Volume':'KSI_Vol'}).round(2)

# 나스닥 지수
IXIC_df = fdr.DataReader('IXIC','2020-01-01','2024-02-01').reset_index().drop(
    ['Open','High','Low','Adj Close'], axis=1).rename(
        columns={'Close':'IXIC_Clo','Volume':'IXIC_Vol'}).round(2)

def shift_date(row):
    if row['Date'].weekday() == 4:  # 금요일이면
        return row['Date'] + pd.Timedelta(days=3)
    else:
        return row['Date'] + pd.Timedelta(days=1)

IXIC_df['Date'] = IXIC_df.apply(shift_date, axis=1)

data_df = [KSI_df,IXIC_df]
dataset_df = reduce(lambda x,y : pd.merge(x,y,on='Date'),data_df)

# 주식시장 개장일만 분류
filtered_df = filter_df(dataset_df)